In [1]:
#если нет pip в конде, то раскомментить строку ниже
#%conda install pip
# %pip install -r requirements.txt

Импорт библиотек

In [2]:
import pandas as pd
import json
import pathlib
import numpy as np

Извлечение путя данных

In [3]:
hack_data_path = pathlib.Path().cwd().parent/'Data'
hack_data_path

PosixPath('/home/sergey/Рабочий стол/Coding/hacathon_mlbg/Data')

Взглянем на данные

In [4]:
# уберем лимит на ширину строк
pd.set_option('display.max_colwidth', None)

In [5]:
# тестовые данные
test_df = pd.read_csv(hack_data_path/'test.csv')

In [6]:
# тренировочные данные
train_df = pd.read_csv(hack_data_path/'train.csv')

In [7]:
# справочник вакансий
okz_df = pd.read_csv(hack_data_path/'okz_3_4_professions.csv', delimiter='\t')

Распакуем json файлы

In [8]:
def json_processor(path):
    json_df = pd.read_json(path)
    json_df['Обязанности'] = [' '.join(el.get('Обязанности', np.nan)) if el.get('Обязанности', np.nan) is not np.nan else np.nan for el in json_df['Content'].values]
    json_df.drop(['Content', 'Position'], axis=1, inplace=True)
    return json_df

In [9]:
vd1 = json_processor(hack_data_path/'vacancy_descriptions'/'1_parsed.json')
vd2 = json_processor(hack_data_path/'vacancy_descriptions'/'2_parsed.json')
vd3 = json_processor(hack_data_path/'vacancy_descriptions'/'3_parsed.json')
vd4 = json_processor(hack_data_path/'vacancy_descriptions'/'4_parsed.json')
vd5 = json_processor(hack_data_path/'vacancy_descriptions'/'5_parsed.json')

In [10]:
vd = pd.concat([vd1, vd2, vd3, vd4, vd5], ignore_index=True)

In [11]:
# сливаем тренировочные данные
train_df_w_description = train_df.merge(vd, how='left', left_on='index', right_on='ID')
train_df_w_description.drop(['ID', 'name'], inplace=True, axis=1)

In [12]:
# сливаем тестовые данные
test_df_w_description = test_df.merge(vd, how='left', left_on='index', right_on='ID')
test_df_w_description.drop(['ID', 'name'], inplace=True, axis=1)

In [13]:
# извлечем обязанности из справочника вакансий
okz_df.drop(['control_number', 'profession'], axis=1, inplace=True)

In [14]:
# функция для обработки описания
import re
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer

def transofrm_description(initial_desciption, morph = MorphAnalyzer(), stop_words = stopwords.words('russian')):
    sample_description_v1 = initial_desciption.lower()
    sample_description_v2 = sample_description_v1[sample_description_v1.find('обязанности'):sample_description_v1.find('.', sample_description_v1.find('обязанности'))]
    sample_description_v3 = sample_description_v2[sample_description_v2.find(':')+1:]
    sample_description_v4 = sample_description_v3.replace('\n', ' ').strip()
    patt1 = re.compile(r'[а-я\s\;]')
    sample_description_v5 = re.findall(patt1, sample_description_v4)
    sample_description_v6 = ''.join(sample_description_v5)
    sample_description_v7 = sample_description_v6.split('; ')
    sample_description_v8 = []
    for i in sample_description_v7:
        # print(f'Изначальный вариант: {i}\n')
        wout_stop_words = [word for word in i.split(' ') if word not in stop_words]
        # print(f'Без стоп слов: {wout_stop_words}\n')
        inf_forms = [morph.parse(word)[0].normal_form for word in wout_stop_words]
        # print(f'Начальная форма: {inf_forms}\n')
        sample_description_v8.append(' '.join(inf_forms))
    sample_description_final = ' '.join(sample_description_v8)
    return sample_description_final
    

In [15]:
# делаем столбец описания в словах у справочника
okz_df['redacted discription'] = okz_df.apply(lambda row: transofrm_description(row['description']), axis=1)

In [16]:
okz_df.drop('description', axis=1, inplace=True)

In [17]:
# количество пустых обязанностей
sum(train_df_w_description['Обязанности'].isna())

4431

In [18]:
train_w_nan_resp = train_df_w_description[train_df_w_description['Обязанности'].isna()]

In [19]:
def nan_transformer(sample_description, stop_words = stopwords.words('russian'), morph = MorphAnalyzer()):
    if sample_description is np.nan:
        return np.nan
    sample_description_v1 = sample_description.lower()
    patt1 = re.compile(r'[а-я\s\;]')
    sample_description_v2 = ''.join(re.findall(patt1, sample_description_v1)).replace('\n', '')
    sample_description_v3 = ' '.join([word for word in sample_description_v2.split(' ') if word not in stop_words])
    inf_forms = ' '.join([morph.parse(word)[0].normal_form for word in sample_description_v3.split()])
    finale = [word for word in inf_forms.split() if len(word)<20]
    return ' '.join(finale)

In [20]:
train_w_nan_resp['Обязанности'] = train_w_nan_resp.apply(lambda row: nan_transformer(row['description']), axis=1)

/tmp/ipykernel_105453/2964548059.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_w_nan_resp['Обязанности'] = train_w_nan_resp.apply(lambda row: nan_transformer(row['description']), axis=1)


In [21]:
train_w_nan_resp.rename(columns={'Обязанности': 'redacted description'}, inplace=True)

/tmp/ipykernel_105453/62153877.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_w_nan_resp.rename(columns={'Обязанности': 'redacted description'}, inplace=True)


In [22]:
def simple_transformer(sample_description, stop_words = stopwords.words('russian'), morph = MorphAnalyzer()):
    if sample_description is np.nan:
        return np.nan
    patt1 = re.compile(r'[а-я\s]')
    sample_description_v2 = ''.join(re.findall(patt1, sample_description.lower())).replace('\n', '')
    sample_description_v3 = ' '.join([word for word in sample_description_v2.split(' ') if word not in stop_words])
    inf_forms = ' '.join([morph.parse(word)[0].normal_form for word in sample_description_v3.split()])
    return inf_forms

In [23]:
train_df_w_description['redacted description'] = train_df_w_description.apply(lambda row: simple_transformer(row['Обязанности']), axis=1)

In [24]:
train_df_w_description.drop(['description', 'Обязанности'], axis=1, inplace=True)

In [25]:
train_w_nan_resp.drop(['description'], axis=1, inplace=True)

/tmp/ipykernel_105453/532034478.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_w_nan_resp.drop(['description'], axis=1, inplace=True)


In [26]:
final_train = train_df_w_description.fillna(train_w_nan_resp)

In [27]:
final_train.dropna(inplace=True)

в обучение okz_df, final_train

In [28]:
# обработаем тестовое задание
save_test = test_df_w_description[test_df_w_description['Обязанности'].isna()]
process_test = test_df_w_description.dropna()

In [29]:
process_test['redacted description'] = process_test.apply(lambda row: simple_transformer(row['Обязанности']), axis=1)

/tmp/ipykernel_105453/1511708657.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  process_test['redacted description'] = process_test.apply(lambda row: simple_transformer(row['Обязанности']), axis=1)


In [30]:
process_test.drop(['Обязанности', 'description'], axis=1, inplace=True)

/tmp/ipykernel_105453/104347109.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  process_test.drop(['Обязанности', 'description'], axis=1, inplace=True)


In [31]:
save_test['redacted description'] = save_test.apply(lambda row: nan_transformer(row['description']), axis=1)

/tmp/ipykernel_105453/4038635796.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  save_test['redacted description'] = save_test.apply(lambda row: nan_transformer(row['description']), axis=1)


In [32]:
save_test.drop(['Обязанности', 'description'], axis=1, inplace=True)

/tmp/ipykernel_105453/2196103870.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  save_test.drop(['Обязанности', 'description'], axis=1, inplace=True)


In [33]:
final_test = pd.concat([process_test, save_test], ignore_index=True)

Обработанные датасеты

*   final_test
*   finale_train
*   okz_df

In [34]:
final_train.drop('index', axis=1, inplace=True)

Машинное обучение

In [35]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.model_selection import train_test_split

Разделяем немаркированные данные и маркированные

In [36]:
train_unmarked = final_train[final_train['target']==-1]
train_marked = final_train[final_train['target']!=-1]

Сливаем справочник и размеченные тренировочные данные

In [37]:
okz_df.rename(columns={'redacted discription':'text'}, inplace=True)

In [38]:
train_marked.rename(columns={'target':'code', 'redacted description':'text'}, inplace=True)

/tmp/ipykernel_105453/1075917921.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_marked.rename(columns={'target':'code', 'redacted description':'text'}, inplace=True)


In [39]:
full_training = pd.concat([okz_df, train_marked], ignore_index=True)

In [40]:
full_training

,code,text
0,111,руководство участие работа законодательный орган административный совет правительство законодательный собрание орган местный самоуправление общественный организация служба государственный административный орган официальный комитет расследование вопрос касаться интерес общество целое затрагивать интерес избиратель разработка консультирование вопрос проведение государственный политика исполнение бюджет законодательство нормативный обеспечение установление цель организация разработка утверждение оценка программа политика процедура реализация рекомендация анализ оценка утверждение документ информационный записка представить доклад обеспечение надлежащий система процедура который разработать реализоваться обеспечение контроль исполнение бюджет выделение использование государственный земля другой ресурс выполнение церемониальный обязанность представительство имя государство правительство организация сообщество официальный мероприятие встреча переговоры собрание общественный слушание
1,1111,участие заседание федеральный региональный орган законодательный власть рассмотрение проект законодательный акт работа комитет комиссия палата работа согласительный специальный комиссия также рабочий группа внесение парламентский запрос депутатский запрос обращение вопрос руководство исполнительный орган власть контроль исполнение они законодательный акт обращение соответствовать должностной лицо требование принять мера немедленный пресечение обнаружиться нарушение право гражданин изучение вопрос представлять интерес общественность защита интерес избиратель который представлять исполнение представительский функция проведение встреча направить оказание услуга сообщество изучение запрос различный группа население гражданин ведение переговоры другой законодатель представитель заинтересованный группа примирение различный интерес выработка политика достижение общественный согласие
2,1112,разработка контроль исполнение государственный бюджет проведение единый финансовый кредитный денежный политика разработка мера проведение единый государственный политика область культура наука образование здравоохранение социальный обеспечение экология осуществление мера обеспечение оборона страна государственный безопасность реализация внешний политика законность право свобода гражданин охрана собственность общественный порядок борьба преступность разработка стратегический цель министерство ведомство соответствие законодательство реализовать политика планирование координация контроль осуществление разработка утверждение оценка программа процедура реализация государственный политика целое соответствие предмет ведение подготовка утверждение ведомственный нормативный правовой акт анализ оценка утверждение методический документ информационный записка доклад ход исполнение государственный ведомственный иной программа обеспечение надлежащий система процедура который разработать реализоваться обеспечение контроль исполнение бюджет координация деятельность другой высокий правительственный служащий должностной лицо надзор реализация государственный политика соблюдение законодательство юридический лицо гражданин
3,1113,управление муниципальный собственность формирование утверждение обеспечение исполнение местный бюджет установление местный налог сбор осуществление охрана общественный порядок представительство свой поселение высокий орган государственный власть информирование местный сообщество государственный норма правило
4,1114,определение формулирование политика норма правило организация осуществление планирование управление координация общий функционирование организация обзор деятельность результат деятельность организация предоставление отчётность совет директор руководящий орган членский организация финансовый учреждение проведение переговоры имя организация член соответствующий группа особый интерес соблюдение интерес организация член соответствующий группа особый интерес законодательный власть правительство широкий общественность планирование организаци

Тренируем модель

In [41]:
x_train, x_test, y_train, y_test = train_test_split(full_training['text'], full_training['code'])

In [42]:
vectorizer = TfidfVectorizer()
tf_train = vectorizer.fit_transform(x_train)
tf_test = vectorizer.transform(x_test)

In [43]:
pass_agg = PassiveAggressiveClassifier()
pass_agg.partial_fit(tf_train, y_train, classes=okz_df['code'])

PassiveAggressiveClassifier()

In [44]:
# точность
pass_agg.score(tf_test, y_test)

0.6763762034065662

Предскажем профессии для неразмеченных данных

In [45]:
train_unmarked.drop('target', axis=1, inplace=True)

/tmp/ipykernel_105453/221119137.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_unmarked.drop('target', axis=1, inplace=True)


In [46]:
x_predict = train_unmarked['redacted description']

In [47]:
tf_predict = vectorizer.transform(x_predict)

In [48]:
y_predict = pass_agg.predict(tf_predict)

Тренируем модель на неразмеченных данных

In [49]:
pass_agg.partial_fit(tf_predict, y_predict)

PassiveAggressiveClassifier()

Полная тренировка на всех данных

In [61]:
train_unmarked['code'] = y_predict

/tmp/ipykernel_105453/249392443.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_unmarked['code'] = y_predict


In [65]:
full_training

,code,text
0,111,руководство участие работа законодательный орган административный совет правительство законодательный собрание орган местный самоуправление общественный организация служба государственный административный орган официальный комитет расследование вопрос касаться интерес общество целое затрагивать интерес избиратель разработка консультирование вопрос проведение государственный политика исполнение бюджет законодательство нормативный обеспечение установление цель организация разработка утверждение оценка программа политика процедура реализация рекомендация анализ оценка утверждение документ информационный записка представить доклад обеспечение надлежащий система процедура который разработать реализоваться обеспечение контроль исполнение бюджет выделение использование государственный земля другой ресурс выполнение церемониальный обязанность представительство имя государство правительство организация сообщество официальный мероприятие встреча переговоры собрание общественный слушание
1,1111,участие заседание федеральный региональный орган законодательный власть рассмотрение проект законодательный акт работа комитет комиссия палата работа согласительный специальный комиссия также рабочий группа внесение парламентский запрос депутатский запрос обращение вопрос руководство исполнительный орган власть контроль исполнение они законодательный акт обращение соответствовать должностной лицо требование принять мера немедленный пресечение обнаружиться нарушение право гражданин изучение вопрос представлять интерес общественность защита интерес избиратель который представлять исполнение представительский функция проведение встреча направить оказание услуга сообщество изучение запрос различный группа население гражданин ведение переговоры другой законодатель представитель заинтересованный группа примирение различный интерес выработка политика достижение общественный согласие
2,1112,разработка контроль исполнение государственный бюджет проведение единый финансовый кредитный денежный политика разработка мера проведение единый государственный политика область культура наука образование здравоохранение социальный обеспечение экология осуществление мера обеспечение оборона страна государственный безопасность реализация внешний политика законность право свобода гражданин охрана собственность общественный порядок борьба преступность разработка стратегический цель министерство ведомство соответствие законодательство реализовать политика планирование координация контроль осуществление разработка утверждение оценка программа процедура реализация государственный политика целое соответствие предмет ведение подготовка утверждение ведомственный нормативный правовой акт анализ оценка утверждение методический документ информационный записка доклад ход исполнение государственный ведомственный иной программа обеспечение надлежащий система процедура который разработать реализоваться обеспечение контроль исполнение бюджет координация деятельность другой высокий правительственный служащий должностной лицо надзор реализация государственный политика соблюдение законодательство юридический лицо гражданин
3,1113,управление муниципальный собственность формирование утверждение обеспечение исполнение местный бюджет установление местный налог сбор осуществление охрана общественный порядок представительство свой поселение высокий орган государственный власть информирование местный сообщество государственный норма правило
4,1114,определение формулирование политика норма правило организация осуществление планирование управление координация общий функционирование организация обзор деятельность результат деятельность организация предоставление отчётность совет директор руководящий орган членский организация финансовый учреждение проведение переговоры имя организация член соответствующий группа особый интерес соблюдение интерес организация член соответствующий группа особый интерес законодательный власть правительство широкий общественность планирование организаци

In [67]:
train_unmarked.rename(columns={'redacted description':'text'}, inplace=True)

/tmp/ipykernel_105453/1446441042.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_unmarked.rename(columns={'redacted description':'text'}, inplace=True)


In [68]:
train_unmarked

,text,code
0,работа касса открытие закрытие смена вежливый работа гость контроль чистота зал знание пк контроль отдача готовый блюдо,5131
1,выкладка товар соответствие планограмма консультирование покупатель косметический продукция работа касса кассовый документация поддержание чистота порядок магазин контроль сохранность товар,5223
3,организация производство работа кровля оформление наряддопуск ведение исполнительнотехнический документация журнал работа получение тмц склад,3123
4,обязанность обслуживание покупатель касса наличный безналичный расчёт выкладка товар зал отслеживание срок годность товар проверка наличие ценник консультация покупатель зал требование ответственность внимательность условие график работа возможный неполный рабочий день неполный рабочий неделя возможность подработка работа совместительство социальный гарантия оплата отпуск больничный лист работа рядом дом скидка магазин сеть фирменный спецодежда возможность профессиональный карьерный рост корпоративный обучение корпоративный мероприятие конкурсымедицинский страховка формат телемедициныдоплата стаж,5223
10,полный ведение бухгалтерский налоговый кадровый учёт платёж клиент банк выставление акт выполнить работа услуга автомойкиконтроль дебиторский кредиторский задолженностикассакомпания предоставлять полный социальный пакет официальный трудоустройство работа удалённый свободный график заработный плата оговариваться собеседование,2411
...,...,...
29990,планирование координация ведение рабочий графика календарь руководитель участие переговоры деловой встреча ведение протокол организация совещание корпоративный мероприятие поддержка руководитель выполнение поручение руководитель работа документооборот контроль исполнение поручение руководитель осуществление сбор материал информация необходимый руководитель умение слышать руководитель проявлять инициатива быстро реагировать непредвиденный ситуация знание английский язык необязательно приветствоваться,4120
29992,выполнение обязанность соответствие должностной инструкция,8343
29994,заполнение письмо отправка почтовый корреспонденция приём документация клиент встреча клиент ведение документооборот компания оформление заказ канцтовары вода содержание офис чистота иной поручение руководитель,3341
29996,группа компания ооо обязанностиюметобязанность требоваться разнорабочий дворникподметание территория улицесбор вынос мусор мелкий косметический ремонтпосадка полить цвет покос трава сезон график работа договорённость обязательно полный рабочий день оформление тк рф зп банковский карта аванс зп оклад вычет ежемесячно,9622


In [69]:
fullest_training = pd.concat([full_training, train_unmarked[['code', 'text']]], ignore_index=True)
fullest_training

,code,text
0,111,руководство участие работа законодательный орган административный совет правительство законодательный собрание орган местный самоуправление общественный организация служба государственный административный орган официальный комитет расследование вопрос касаться интерес общество целое затрагивать интерес избиратель разработка консультирование вопрос проведение государственный политика исполнение бюджет законодательство нормативный обеспечение установление цель организация разработка утверждение оценка программа политика процедура реализация рекомендация анализ оценка утверждение документ информационный записка представить доклад обеспечение надлежащий система процедура который разработать реализоваться обеспечение контроль исполнение бюджет выделение использование государственный земля другой ресурс выполнение церемониальный обязанность представительство имя государство правительство организация сообщество официальный мероприятие встреча переговоры собрание общественный слушание
1,1111,участие заседание федеральный региональный орган законодательный власть рассмотрение проект законодательный акт работа комитет комиссия палата работа согласительный специальный комиссия также рабочий группа внесение парламентский запрос депутатский запрос обращение вопрос руководство исполнительный орган власть контроль исполнение они законодательный акт обращение соответствовать должностной лицо требование принять мера немедленный пресечение обнаружиться нарушение право гражданин изучение вопрос представлять интерес общественность защита интерес избиратель который представлять исполнение представительский функция проведение встреча направить оказание услуга сообщество изучение запрос различный группа население гражданин ведение переговоры другой законодатель представитель заинтересованный группа примирение различный интерес выработка политика достижение общественный согласие
2,1112,разработка контроль исполнение государственный бюджет проведение единый финансовый кредитный денежный политика разработка мера проведение единый государственный политика область культура наука образование здравоохранение социальный обеспечение экология осуществление мера обеспечение оборона страна государственный безопасность реализация внешний политика законность право свобода гражданин охрана собственность общественный порядок борьба преступность разработка стратегический цель министерство ведомство соответствие законодательство реализовать политика планирование координация контроль осуществление разработка утверждение оценка программа процедура реализация государственный политика целое соответствие предмет ведение подготовка утверждение ведомственный нормативный правовой акт анализ оценка утверждение методический документ информационный записка доклад ход исполнение государственный ведомственный иной программа обеспечение надлежащий система процедура который разработать реализоваться обеспечение контроль исполнение бюджет координация деятельность другой высокий правительственный служащий должностной лицо надзор реализация государственный политика соблюдение законодательство юридический лицо гражданин
3,1113,управление муниципальный собственность формирование утверждение обеспечение исполнение местный бюджет установление местный налог сбор осуществление охрана общественный порядок представительство свой поселение высокий орган государственный власть информирование местный сообщество государственный норма правило
4,1114,определение формулирование политика норма правило организация осуществление планирование управление координация общий функционирование организация обзор деятельность результат деятельность организация предоставление отчётность совет директор руководящий орган членский организация финансовый учреждение проведение переговоры имя организация член соответствующий группа особый интерес соблюдение интерес организация член соответствующий группа особый интерес законодательный власть правительство широкий общественность планирование организаци

In [147]:
il_data = pd.read_csv(hack_data_path/'train_fd.csv')

In [159]:
x_train1, x_test1, y_train1, y_test1 = train_test_split(fullest_training['text'], fullest_training['code'], test_size=0.15)
vectorizer1 = TfidfVectorizer()
tf_train1 = vectorizer1.fit_transform(x_train1)
tf_test1 = vectorizer1.transform(x_test1)
pas_ag1 = PassiveAggressiveClassifier(max_iter=50000)
pas_ag1.fit(tf_train1, y_train1)

PassiveAggressiveClassifier(max_iter=50000)

In [160]:
pas_ag1.score(tf_test1, y_test1)

0.7691468470434213

In [161]:
final_results = final_test['index']
final_feeding = final_test['redacted description']

In [162]:
tf_pred1 = vectorizer1.transform(final_feeding)
prof = pas_ag1.predict(tf_pred1)
prof

array([8322, 3323, 5223, ..., 7212, 9333, 4222])

In [163]:
f_pr = pd.Series(prof)
f_pr.name = 'target'

In [164]:
finale1 = pd.concat([final_results, f_pr], axis=1)
save_path = hack_data_path.parent/'Results'
finale1.to_csv(save_path/'submission_pass_aggr_v4.csv', index=False)

Предсказываем тестовые данные

In [50]:
final_test

,index,redacted description
0,26461447,выполнение поступать заявка приложение контроль технический состояние автомобиль чистота порядок салон соблюдение стандарт компания
1,26464220,поиск поставщик ведение перевозка начало конец
2,26467473,обслуживание покупатель торговый зал зона примерочный кассовый зона выкладка довесок товар торговый зал совершенствование свой знание товар поддержание порядок чистота рабочий место
3,26468989,оформление приём перевод увольнение сотрудник отчётность сзвтд оформление личный дело заполнение учёт хранение трудовой книжка выдача справка настоящий прошлый трудовой деятельность оформление отпуск сотрудник график оформление командировка составление приказ ведение табель учёт рабочий время оформление больничный лист разработка лна правило внутренний распорядок положение отчёт отчётность численность состав работник защита персональный данные так учёт военнообязанный поиск подбор персонал
4,26471705,обработка входящий горячий заявка клиент первичный консультирование клиент телефон поиск запись гражданин очный приём профильный юрист компания весь работа телефон обучение процесс работа выполнение поручение руководитель
...,...,...
1085,33916809,контроль ход выполнение работа взаимодействие строительный техника срок соответствие сграфик производство работ;составленией подача заявка контроль поступленияматериал выполнение работа хранение изделие складированиеучёт контроль техника безопасность итрудовой дисциплина выполнить работа ведение журнал ведение сдача объектов;опытработа сфера строительство работас металлоконструкция бетонныеработа инженерный сеть планировказдание участок внутренний отделкаумение пк знание ичтение чертёж смет;знаниеспособ ведение строительный работстроительный норма коллектив рабочихусловие уровеньзаработный плата каждый заработный плата
1086,113912590,год являться один крупный поставщик посуда сувенирный изделие предмет домашний интерьер быт россия продажа наименование продукция произвести группа компания хороший фабрика китай россия европа больший часть товар произвести собственный торговый марка бренд один первый заняться производство реализация лицензионный товар опыт накопить область позволить мы стать лицензиат зимний олимпийский игра чемпионат мир футбол свой сотрудник ценить профессионализм нацеленность результат каждый понимать зависеть успешность эффективность весь компания вместе совершенствоваться каждый день связь расширение искать наш команда менеджер работа сеть обязанность развитие отношение региональный федеральный сеть поиск привлечение новый клиент партнёр вхождение новый крупный торговый сеть расширение дистрибьюция оптимизация условие сотрудничество ценовой политика обеспечение условие хороший представленность продукция ввод новый матрица работать сеть ротация ассортимент анализ активность конкурент анализ статистика продажа сеть планирование индивидуальный объём сбыт индивидуальный целевой показатель развитие активный клиентский база ведение переговоры уровень руководитель компания условие продажа продукция порядок проведение расчёт составление продавать коммерческий предложение проведение работа стимулирование продажа реализация специальный программа скидка бонус другой сбытовый акция выполнение план увеличение продажа возможный командировка участие выставка требование реальный успешный опыт продажа менеджер работа сеть менеджер работа ключевой клиент опыт продажа х год опыт работа локальный федеральный сеть опыт работа сегмент посуда подарок товар дом умение расставлять приоритет реагировать изменение ситуация умение вести переговоры добиваться поставить цель умение работать режим многозадачность высокий организаторский способность условие работа стабильный компания лидер рынок свой сегмент конкурентный заработный плата оклад совокупный доход р возможность карьерный рост график работа либо официальный трудоустройство место работа г королева ул калинин реальный успешный опыт продажа менеджер работа сеть менеджер работа ключевой клиент опыт продажа развитие от

Финальное предсказание

In [89]:
final_results = final_test['index']
final_feeding = final_test['redacted description']

In [52]:
tf_final_predict = vectorizer.transform(final_feeding)

In [53]:
professions = pass_agg.predict(tf_final_predict)

In [54]:
professions

array([8322, 8322, 5223, ..., 7212, 9333, 4222])

In [55]:
final_prediction = pd.Series(professions)
final_prediction

0       8322
1       8322
2       5223
3       4120
4       4222
        ... 
1085    3123
1086    1221
1087    7212
1088    9333
1089    4222
Length: 1090, dtype: int64

In [ ]:
final_results.drop('code', inplace=True)

In [57]:
final_results

0        26461447
1        26464220
2        26467473
3        26468989
4        26471705
          ...    
1085     33916809
1086    113912590
1087     33948550
1088     35926971
1089    166397492
Name: index, Length: 1090, dtype: int64

In [58]:
final_prediction.name = 'target'

In [59]:
final_prediction

0       8322
1       8322
2       5223
3       4120
4       4222
        ... 
1085    3123
1086    1221
1087    7212
1088    9333
1089    4222
Name: target, Length: 1090, dtype: int64

In [60]:
finale = pd.concat([final_results, final_prediction], axis=1)
finale

,index,target
0,26461447,8322
1,26464220,8322
2,26467473,5223
3,26468989,4120
4,26471705,4222
...,...,...
1085,33916809,3123
1086,113912590,1221
1087,33948550,7212
1088,35926971,9333


In [91]:
# путь сохранения
save_path = hack_data_path.parent/'Results'
save_path

PosixPath('/home/sergey/Рабочий стол/Coding/hacathon_mlbg/Results')

In [93]:
finale.to_csv(save_path/'submission_pass_aggr_v1.csv', index=False)